In [1]:
from twython import Twython as tw
from twython import TwythonError

In [2]:
# App keys and secrets stored as environment variables
import os
APP_KEY = os.environ.get('TWITTER_APP_KEY')
APP_SECRET = os.environ.get('TWITTER_APP_SECRET')

Access_Token = os.environ.get('TWITTER_ACCESS_TOKEN')
Access_Secret = os.environ.get('TWITTER_ACCESS_SECRET')

twitter = tw(APP_KEY, APP_SECRET, Access_Token, Access_Secret)

In [4]:
import pandas as pd

In [5]:
# Define a query

bitcoin_query = {'q': 'bitcoin',  
        'result_type': 'popular',
        'count': 100,
        'lang': 'en',
        }

In [6]:
dict_ = {'user': [], 'date': [], 'text': [], 'favorite_count': [], 'user_ID': []}  
for status in twitter.search(**bitcoin_query)['statuses']:  
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])
    dict_['user_ID'].append(status['user'])

TwythonAuthError: Twitter API returned a 400 (Bad Request), Bad Authentication data.

In [6]:
df = pd.DataFrame(dict_)  
df.sort_values(by='favorite_count', inplace=True, ascending=False)  

In [7]:
#Remove duplicates to find individual users

df = df.drop_duplicates(subset='user', keep='first', inplace=False)
df.head()

,date,favorite_count,text,user,user_ID
3,Sat Dec 01 19:35:10 +0000 2018,10971,I remember when so many people berated me when...,RealJamesWoods,"{'id': 78523300, 'id_str': '78523300', 'name':..."
0,Sun Dec 02 17:43:55 +0000 2018,4913,Bitcoin facts the media ignores:\n\n- 70%+ of ...,APompliano,"{'id': 339061487, 'id_str': '339061487', 'name..."
2,Sun Dec 02 21:45:07 +0000 2018,1057,Fidelity Investments is looking to expand its ...,coindesk,"{'id': 1333467482, 'id_str': '1333467482', 'na..."
4,Mon Dec 03 00:40:56 +0000 2018,1047,Crypto markets do not shut down for the pagean...,ErikVoorhees,"{'id': 61417559, 'id_str': '61417559', 'name':..."
13,Mon Dec 03 02:06:46 +0000 2018,409,Congrats to /r/bitcoin for reaching a million ...,lopp,"{'id': 23618940, 'id_str': '23618940', 'name':..."


In [8]:
# Pull user names & ID's and store

influencers = df.loc[:, 'user']

In [9]:
# Storing the newly found users in a list. 

my_user_id = 705469309
my_list_name = 'tweet_smart_list'
# Create dictionary of my lists
my_lists_dict = twitter.show_owned_lists(user_id = my_user_id) 
#Split so that running code doesn't keep sending requests to Twitter. 

In [18]:
my_lists_df = pd.DataFrame(my_lists_dict['lists']) 
names = my_lists_df.loc[my_lists_df.name == my_list_name]
my_list_id = int(names.id)
list_slug = names.slug

In [19]:
for user in influencers:
    try:
        twitter.add_list_member(list_id = my_list_id, screen_name=user)
    except TwythonError as e:
        print(e)

        